In [1]:
import torch
from torch import nn, optim

import torch.nn.functional as F
from einops import rearrange
from typing import Optional
from ember import (
    Transformer,
    MultiHeadLatentAttn,
    KVCache,
    GroupedQueryAttn,
    Tokenizer,
    RoPE,
    apply_rotary_pos_emb,
    TopKSampler
)
from transformers import AutoTokenizer

/Users/ryanpegoud/Documents/Projects/ember/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
B, S, D = 8, 1024, 512
mla_kwargs = {
    "latent_dim": 256,
    "pos_dim": 128,
    "n_heads": 16,
}
gqa_kwargs = {
    "n_query_heads": 8,
    "n_query_groups": 4,
}
# 
# tk = Tokenizer()
tk = Tokenizer(model = "openai-community/gpt2")
model = Transformer(
    vocab_size=len(tk.base),
    model_dim=D,
    hidden_dim=1024,
    attn_module=GroupedQueryAttn,
    attn_kwargs=gqa_kwargs,
    n_attn_blocks=3,
)

# mini_deepseek = Transformer(
#     vocab_size=tk.vocab_size,
#     model_dim=D,
#     hidden_dim=1024,
#     attn_module=MultiHeadLatentAttn,
#     attn_kwargs=mla_kwargs,
#     n_attn_blocks=3,
# )

<bound method Module.parameters of Transformer(
  (embed): Embedding(50258, 512)
  (attn_blocks): ModuleList(
    (0-2): 3 x AttentionBlock(
      (mlp): SwiGLU(
        (W): Linear(in_features=512, out_features=682, bias=False)
        (V): Linear(in_features=512, out_features=682, bias=False)
        (W2): Linear(in_features=682, out_features=512, bias=False)
      )
      (norm): RMSNorm()
      (attn): GroupedQueryAttn(
        (fused_qkv): Linear(in_features=512, out_features=1024, bias=True)
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
        (rope): RoPE()
      )
    )
  )
  (norm): RMSNorm()
)>
Parameter count: 3.12e+07


In [3]:
txt = ["Hey there", "How are you?"]
x = tk(txt, mode="inference")
tk.decode(x)

['<|pad|><|pad|>Hey there', 'How are you?']

In [4]:
out = model.generate(
    txt,
    max_new_tokens=50,
    sampler=TopKSampler(50),
    tokenizer=tk,
)
out

['<|pad|><|pad|>Hey thereHello Pand Pand ruled salvationlayist authority833ired dispatchedBand Essentially glacGl authorityCheck smellLabellayCheck Ulster Telegraph states glacistaila occupy gotten Goodman Musk sparkist ruled anarchism gottenBand assemb dosage Telegraph Tact ReturningpillarCheck spark spark uncond prod ratherired Godzilla',
 'How are you?earableearable Ga DWHuman lawmakersmiah Glovesmiahsenperial braggingtaining speakers discriminating thirteen claws revolver pioneeringaterial loader sentencing abras� unreal stupidity abras� townsmonthssafemiah discriminating Iv abrasッドα Iv pioneeringッドmonths discriminating lawmakers unreal braggingperialrep HAαenn �']